# Python Pre-Processing


# Import Drive Location of JSON File

In [ ]:
# Import Google Drive For JSON File
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Import of JSON File

In [ ]:
# Import JSON file for Pre-processing
import json

file_path = '/content/drive/MyDrive/PreProcessedData2.json'
with open(file_path, 'r') as file:
    data = json.load(file)


# Load JSON into DataFrame

In [ ]:
import pandas as pd
import json
# Load the JSON file into a DataFrame
df = pd.read_json('/content/drive/MyDrive/PreProcessedData2.json')


In [ ]:

# Parse the 'json_data' column
df['json_data'] = df['json_data'].apply(json.loads)
df = pd.json_normalize(df['json_data'])

# Define Sensor Ranges

In [ ]:
# Define the sensor range categories based on the table
def categorize_sensor(value):
    if 0 <= value <= 8:
        return 'fridge/cold rooms'
    elif -25 <= value <= -10:
        return 'freezer'
    elif 35 <= value <= 40:
        return 'incubator'
    elif -85 <= value <= -70:
        return 'cryogenic'
    elif -200 <= value <= -190:
        return 'liquid nitrogen'
    else:
        return 'other'


# Apply the Categorization Function to the DataFrame & Display Total Records


In [ ]:
# Apply the categorization function to the 'channel_value' column
df['category'] = df['channel_value'].apply(categorize_sensor)

# Calculate and display total number of records
total_records = len(df)
print(f"Total records in the dataset: {total_records}")


Total records in the dataset: 1134937


# Calculate Total Number Of Records & Categories

In [ ]:
# Calculate the total number of records
total_records = len(df)

# Count the number of records in each category
category_counts = df['category'].value_counts()

# Calculate the number of unique sensors for each category
unique_sensors = df.groupby('category')['channel_description'].nunique()


# Create a DataFrame for the summary
category_summary = pd.DataFrame({
    'Category': category_counts.index,
    'Total Records': [total_records] * len(category_counts),
    'Category Records': category_counts.values,
    'Unique Sensors': unique_sensors.reindex(category_counts.index).values,
    'Category %': (category_counts / total_records * 100).values
})

# Print the summary DataFrame
print(category_summary)

# Create a dictionary to hold the unique sensor names for each category
unique_sensors_names = df.groupby('category')['channel_description'].apply(set).to_dict()

# Print the names of the unique sensors for each category
for category, sensors in unique_sensors_names.items():
    print(f"\nCategory: {category}")
    print(f"Unique sensors: {', '.join(sensors)}")



            Category  Total Records  Category Records  Unique Sensors  \
0  fridge/cold rooms        1134937            807231               9   
1            freezer        1134937            182872               3   
2          cryogenic        1134937            115158               1   
3              other        1134937             29676               8   

   Category %  
0   71.125622  
1   16.112965  
2   10.146643  
3    2.614771  

Category: cryogenic
Unique sensors: CTGeneFrz

Category: freezer
Unique sensors: CT-20FrzA, CTGeneFrz, Freezer1A

Category: fridge/cold rooms
Unique sensors: CTGeneFrz, CytSupFdA, CT-20FrzA, DisBotFdA, ASUFrid1A, ASUFrdg4P, CDRmFrd A, ClinTr63P, IsoSupFdP

Category: other
Unique sensors: CTGeneFrz, CT-20FrzA, DisBotFdA, ASUFrid1A, ASUFrdg4P, CDRmFrd A, CytSupFdA, IsoSupFdP


# Data Pre-Processing Overview So Far

Initial data pre-processing identified 1,134,937 records, predominantly within the 'Fridge/Cold rooms' range, as depicted in the dataset distribution chart. The 'Other Category' comprises 8 unique sensors, prompting further investigation to differentiate between expected range deviations and sensor data anomalies. Such scrutiny is essential to enhance the dataset's accuracy and ensure its reliability for subsequent analyses.

![Category Percentages](https://drive.google.com/uc?export=download&id=1RD6qRxqghwMAT_-tzqTQX-f5aGyej4-u)
*Figure: Distribution of Temperature Categories across the Dataset*


# Determine Sensor Cateogry Based On Predominant Range

In [ ]:
# Define category ranges
category_ranges = {
    'fridge/cold rooms': (0, 8),
    'freezer': (-25, -10),
    'incubator': (35, 40),
    'cryogenic': (-85, -70),
    'liquid nitrogen': (-200, -190),
}

# Calculate the proportion of readings for each category for each sensor
category_proportions = {sensor: {} for sensor in df['channel_description'].unique()}

for sensor in category_proportions.keys():
    sensor_data = df[df['channel_description'] == sensor]
    total_records = len(sensor_data)

    for category, (lower_limit, upper_limit) in category_ranges.items():
        in_range_count = sensor_data[
            (sensor_data['channel_value'] >= lower_limit) &
            (sensor_data['channel_value'] <= upper_limit)
        ].shape[0]
        category_proportions[sensor][category] = in_range_count / total_records if total_records > 0 else 0

# Determine the predominant category for each sensor
threshold = 0.5  # 50% threshold
predominant_categories = {}

for sensor, proportions in category_proportions.items():
    max_category = max(proportions, key=proportions.get)
    if proportions[max_category] >= threshold:
        predominant_categories[sensor] = max_category
    else:
        predominant_categories[sensor] = 'other'

# Initialize a list to store results
results = []

# Iterate over each sensor and its predominant category
for sensor, category in predominant_categories.items():
    sensor_data = df[df['channel_description'] == sensor]
    total_records = len(sensor_data)
    lower_limit, upper_limit = category_ranges[category]

    # Count in-range and out-of-range records
    in_range_records_count = sensor_data[
        (sensor_data['channel_value'] >= lower_limit) &
        (sensor_data['channel_value'] <= upper_limit)
    ].shape[0]
    out_of_range_records_count = total_records - in_range_records_count

    # Calculate the percentage of records out of range
    percent_out_of_range = (out_of_range_records_count / total_records) * 100 if total_records > 0 else 0

    # Append results
    results.append({
        'Sensor': sensor,
        'Predominant Category': category,
        'Total Records': total_records,
        'In Range Records': in_range_records_count,
        'Out of Range Records': out_of_range_records_count,
        'Percent Out of Range': percent_out_of_range
    })

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Sort the results by 'Out of Range Records' in descending order
sorted_results_df = results_df.sort_values('Out of Range Records', ascending=False)

# Display Sensor Name, Predominant Category, Total Records, In Range Records, Out of Range Records, and Percentage Out of Range
print(sorted_results_df[['Sensor', 'Predominant Category', 'Total Records', 'In Range Records', 'Out of Range Records', 'Percent Out of Range']])


      Sensor Predominant Category  Total Records  In Range Records  \
4  CT-20FrzA              freezer         115084             67497   
7  DisBotFdA    fridge/cold rooms         115368             93489   
0  ASUFrdg4P    fridge/cold rooms         116082            115700   
5  CTGeneFrz            cryogenic         115506            115158   
6  CytSupFdA    fridge/cold rooms         115214            114983   
2  CDRmFrd A    fridge/cold rooms         115460            115268   
9  IsoSupFdP    fridge/cold rooms          95661             95628   
1  ASUFrid1A    fridge/cold rooms         115818            115807   
3  ClinTr63P    fridge/cold rooms         115378            115378   
8  Freezer1A              freezer         115366            115366   

   Out of Range Records  Percent Out of Range  
4                 47587             41.349797  
7                 21879             18.964531  
0                   382              0.329078  
5                   348              

# Analysing Categorization

Recent sensor categorization aligned with Kelsius thresholds revealed high conformity across most sensors. Yet, two sensors—'CT-20FrzA' and 'DisBotFdA'—deviated significantly, with 41% and 19% of their readings respectively outside their main categories. Investigation using a smoothed average temperature graph exposed 'CT-20FrzA's' abnormal temperature fluctuations, hinting at possible relocations and validating its exclusion from standard categorization. 'DisBotFdA' exhibited similar inconsistencies, leading to its exclusion to preserve data integrity. Consequently, freezer and cryogenic sensors, being underrepresented, were removed to maintain dataset balance, crucial for the proposed machine learning models focused on anomaly detection and regression. This dataset refinement is a foundational step toward a comprehensive, focused analysis. Remaining data, set aside for now, will be utilized for future enhancements.

![Category Percentages](https://drive.google.com/uc?export=download&id=1a_A768BJSMYTLXqdD0VkXQOmVYHp70vo)
*Figure: Smoothed Average of CT-20FrzA*

# Final Phase of Pre-Processing

Before final pre-processing, the dataset stands at 673,613 records. Next, we'll refine the categorization by excluding four sensors/categories and any outlying temperature records. This crucial step ensures precise & reliable data allowing for accuracy when applying the machine learning model.

In [ ]:
import pandas as pd

# Drop rows where 'channel_description' matches any of the specified sensors
df_final= df[~df['channel_description'].isin(['CT-20FrzA', 'DisBotFdA', 'CTGeneFrz', 'Freezer1A'])]

# Initialize a dictionary to calculate category proportions for each sensor in the filtered dataset
category_proportions_filtered = {sensor: {} for sensor in df_final['channel_description'].unique()}

# Calculate the proportion of in-range readings for each category for each sensor
for sensor in category_proportions_filtered.keys():
    sensor_data = df_final[df_final['channel_description'] == sensor]
    total_records = len(sensor_data)
    for category, (lower_limit, upper_limit) in category_ranges.items():
        in_range_count = sensor_data[(sensor_data['channel_value'] >= lower_limit) & (sensor_data['channel_value'] <= upper_limit)].shape[0]
        category_proportions_filtered[sensor][category] = in_range_count / total_records if total_records > 0 else 0

# Determine the predominant category for each sensor
predominant_categories_filtered = {}
for sensor, proportions in category_proportions_filtered.items():
    max_category = max(proportions, key=proportions.get)
    predominant_categories_filtered[sensor] = max_category if proportions[max_category] >= threshold else 'other'

# Generate a summary for each sensor in the filtered dataset
results_filtered = []
for sensor, category in predominant_categories_filtered.items():
    sensor_data = df_final[df_final['channel_description'] == sensor]
    total_records = len(sensor_data)
    lower_limit, upper_limit = category_ranges[category]

    in_range_records_count = sensor_data[(sensor_data['channel_value'] >= lower_limit) & (sensor_data['channel_value'] <= upper_limit)].shape[0]
    out_of_range_records_count = total_records - in_range_records_count

    percent_out_of_range = (out_of_range_records_count / total_records) * 100 if total_records > 0 else 0

    results_filtered.append({
        'Sensor': sensor,
        'Predominant Category': category,
        'Total Records': total_records,
        'In Range Records': in_range_records_count,
        'Out of Range Records': out_of_range_records_count,
        'Percent Out of Range': percent_out_of_range
    })

# Convert the results to a DataFrame and sort it
df_final_table = pd.DataFrame(results_filtered)
sorted_final_df = df_final_table.sort_values('Out of Range Records', ascending=False)

# Calculate the sum for each column to create a totals row
totals = {
    'Sensor': 'Total',
    'Predominant Category': 'N/A',
    'Total Records': df_final_table['Total Records'].sum(),
    'In Range Records': df_final_table['In Range Records'].sum(),
    'Out of Range Records': df_final_table['Out of Range Records'].sum(),
    'Percent Out of Range': (df_final_table['Out of Range Records'].sum() / df_final_table['Total Records'].sum()) * 100
}

# Append the totals row to the DataFrame
sorted_final_df = sorted_final_df.append(totals, ignore_index=True)

# Display the final DataFrame
print(sorted_final_df[['Sensor', 'Predominant Category', 'Total Records', 'In Range Records', 'Out of Range Records', 'Percent Out of Range']])



      Sensor Predominant Category  Total Records  In Range Records  \
0  ASUFrdg4P    fridge/cold rooms         116082            115700   
1  CytSupFdA    fridge/cold rooms         115214            114983   
2  CDRmFrd A    fridge/cold rooms         115460            115268   
3  IsoSupFdP    fridge/cold rooms          95661             95628   
4  ASUFrid1A    fridge/cold rooms         115818            115807   
5  ClinTr63P    fridge/cold rooms         115378            115378   
6      Total                  N/A         673613            672764   

   Out of Range Records  Percent Out of Range  
0                   382              0.329078  
1                   231              0.200496  
2                   192              0.166291  
3                    33              0.034497  
4                    11              0.009498  
5                     0              0.000000  
6                   849              0.126037  


<ipython-input-9-704b75e14d9d>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sorted_final_df = sorted_final_df.append(totals, ignore_index=True)


# Final Analysis

Post-preprocessing, the dataset for 'Fridge/Cold Room' stands at 672,764 records, a 40.7% decrease from the original JSON File.

The below graph displays daily average temperatures over a year, with variability shown by the scattered blue dots. The red 'Regression Fit' line indicates a general downward trend in temperature, as per the linear regression model's analysis.


In [ ]:
# @title Regression Graph
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression

# Ensure 'date' column is a datetime dtype before setting it as index
df_final['date'] = pd.to_datetime(df_final['date'])
df_aggregated = df_final.set_index('date').resample('D').mean().reset_index()

# Aggregate data by month
df_aggregated = df_final.set_index('date').resample('D').mean().reset_index()

# Use the ordinal value of the date for regression
df_aggregated['date_ordinal'] = df_aggregated['date'].map(pd.Timestamp.toordinal)

# Prepare the data for the regression model
X_aggregated = df_aggregated[['date_ordinal']]
y_aggregated = df_aggregated['channel_value']

# Fit the regression model
model = LinearRegression()
model.fit(X_aggregated, y_aggregated)

# Create a range for predictions
x_range = np.linspace(X_aggregated['date_ordinal'].min(), X_aggregated['date_ordinal'].max(), 100).astype(int)
y_range = model.predict(x_range.reshape(-1, 1))

# Create the scatter plot for the aggregated data
fig = px.scatter(df_aggregated, x='date', y='channel_value', opacity=0.65, title='Daily Aggregated Temperature Over Time')

# Convert ordinal dates back to datetime for plotting
x_dates = [pd.Timestamp.fromordinal(int(ordinal)) for ordinal in x_range]

# Add the regression fit line
fig.add_traces(go.Scatter(x=x_dates, y=y_range, name='Regression Fit', mode='lines'))

# Show the plot
fig.show()


<ipython-input-10-f1f050ea4b46>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['date'] = pd.to_datetime(df_final['date'])
<ipython-input-10-f1f050ea4b46>:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_aggregated = df_final.set_index('date').resample('D').mean().reset_index()
<ipython-input-10-f1f050ea4b46>:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the functio

In [ ]:
# @title Export To CSV
# Export to csv
df_final.to_csv('/content/drive/My Drive/final_dataset.csv', index=False)
